In [17]:
# python 3
import pandas as pd

In [18]:
nrodada = 13

In [19]:
tabelas = []

for i in range(1,nrodada+1):
    x = pd.read_excel(r'.\Excel\{}rodada.xlsx'.format(str(i)))
    x['rodada'] = i
    tabelas.append(x)

In [20]:
tabela = tabelas[0]
for i in range(1,len(tabelas)):
    tabela = pd.concat([tabela,tabelas[i]],axis=0)

In [21]:
tabela['chave'] = tabela.apply(lambda x: '{}_{}_{}'.format(x['Abreviacao'],x['Apelido'],x['Posicao']),axis=1)

In [22]:
agrupado = tabela.groupby(['chave','rodada']).sum()

In [23]:
textos = [col for col in tabela.columns if tabela[col].dtype == 'O']

In [24]:
campos_dif = ['Gol', 'Assistência', 'Finalização_na_trave', 'Finalização_pra_fora', 'Roubada_de_bola',
              'Passe_errado', 'Falta_cometida', 'Pênalti_perdido', 'Gol_contra', 'Falta_sofrida', 'Impedimento',
              'Cartão_amarelo', 'Cartão_vermelho', 'Defesa_de_pênalti', 'Jogos_sem_sofrer_gols', 'Finalização_defendida',
              'Defesa_difícil', 'Gol_sofrido']

outros_campos = ['Atleta_pontos','Número_Jogos','preco_cartoleta','preco_variacao','Atleta_média'] + textos

## 4 rodada menos a terceira

In [25]:
atual = nrodada
anterior = atual -1

In [26]:
fixo = tabela[tabela.rodada == atual][outros_campos]

In [27]:
resultados = []
for chave in  fixo.chave.tolist():
    try:
        t = pd.DataFrame(agrupado.loc[(chave,atual)][campos_dif] - agrupado.loc[(chave,anterior)][campos_dif])
    except:
        t = pd.DataFrame(agrupado.loc[(chave,atual)][campos_dif])
        
    t=t.transpose()
    t['chave'] = chave
    resultados.append(t)

In [28]:
resultado = resultados[0]
for r in range(1,len(resultados)):
    resultado = pd.concat([resultado,resultados[r]],axis=0)

In [29]:
planilha = pd.merge(fixo,resultado, how='left', on='chave')

In [30]:
ordem = ['Time', 'Abreviacao','Nome','Apelido','Posicao','Número_Jogos','seletor','preco_cartoleta','preco_variacao',
         'Atleta_pontos', 'Atleta_média','Gol','Assistência','Finalização_na_trave','Finalização_pra_fora','Roubada_de_bola',
         'Passe_errado', 'Falta_cometida', 'Pênalti_perdido', 'Gol_contra','Falta_sofrida', 'Impedimento','Cartão_amarelo',
         'Cartão_vermelho','Defesa_de_pênalti', 'Jogos_sem_sofrer_gols', 'Finalização_defendida','Defesa_difícil',
         'Gol_sofrido','Data', 'Ranking','Ranking_Visitante', 'adversario','hora', 'local', 'local2']


planilha = planilha[ordem]

In [31]:
planilha.to_pickle(r'.\Isoladas\{}rodada_isolada.pkl'.format(str(nrodada)))